In [1]:
import os
os.chdir("../")

In [2]:
import os
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from src.logging import logging
from src.utils.utils import *
from src.constants import *

In [16]:
contents = read_yaml(Path(CONFIG_FILE_PATH))
contents

[2025-07-27 04:23:23,780: INFO: utils: yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'data_file': 'data/bank_subscription.xlsx', 'save_file': 'artifacts/data_ingestion/data'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_file': 'data_ingestion/data'}})

In [17]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    data_file: Path
    save_file: Path

In [18]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
    
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            data_file=Path(config.data_file),
            save_file=Path(config.save_file)
        )

        return data_ingestion_config

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def read_data(self) -> pd.DataFrame:
        try:
            df = pd.read_excel(self.config.data_file)
            logging.info(f"Data read successfully from {self.config.data_file}")
            return df
        except Exception as e:
            logging.error(f"Error reading data from {self.config.data_file}: {e}")
            raise e
    
    def save_data(self, df: pd.DataFrame):
        try:
            os.makedirs(self.config.save_file, exist_ok=True)
            file_path = os.path.join(self.config.save_file, "data.csv")
            df.to_csv(file_path, index=False)
            logging.info(f"Data saved successfully to {file_path}")
        except Exception as e:
            logging.error(f"Error saving data to {self.config.save_file}: {e}")
            raise e

In [20]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(data_ingestion_config)
df = data_ingestion.read_data()
data_ingestion.save_data(df)

[2025-07-27 04:23:24,922: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2025-07-27 04:23:24,924: INFO: utils: yaml file: params.yaml loaded successfully]
[2025-07-27 04:23:24,926: INFO: utils: Created Directory at: artifacts]
[2025-07-27 04:23:24,928: INFO: utils: Created Directory at: artifacts/data_ingestion]
[2025-07-27 04:23:29,243: INFO: 1208334827: Data read successfully from data\bank_subscription.xlsx]
[2025-07-27 04:23:29,393: INFO: 1208334827: Data saved successfully to artifacts\data_ingestion\data\data.csv]
